# XGBoost RUL Regressor Training for Predictive Maintenance

This notebook implements XGBoost regression for Remaining Useful Life (RUL) prediction in predictive maintenance.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Preparation for RUL Prediction

In [ ]:
def prepare_rul_data(df, sequence_length=30, rul_column=None):
    """Prepare data for RUL prediction by creating sequences with RUL targets"""
    
    if rul_column is None:
        # Assume RUL is the last column or create synthetic RUL
        if 'RUL' in df.columns:
            rul_column = 'RUL'
        else:
            print("No RUL column found. Creating synthetic RUL based on time.")
            df = df.copy()
            df['RUL'] = len(df) - np.arange(len(df))
            rul_column = 'RUL'
    
    sequences = []
    rul_targets = []
    
    for i in range(len(df) - sequence_length):
        seq = df.iloc[i:i+sequence_length].drop(columns=[rul_column]).values
        rul = df.iloc[i+sequence_length][rul_column]
        sequences.append(seq.flatten())  # Flatten for XGBoost
        rul_targets.append(rul)
    
    X = np.array(sequences)
    y = np.array(rul_targets)
    
    print(f"Created {len(X)} sequences with {X.shape[1]} features each")
    print(f"RUL range: {y.min():.1f} - {y.max():.1f}")
    
    return X, y

def create_rul_features(df, window_sizes=[5, 10, 20]):
    """Create additional features for RUL prediction"""
    df_featured = df.copy()
    
    # Rolling statistics
    for col in df.select_dtypes(include=[np.number]).columns:
        for window in window_sizes:
            df_featured[f'{col}_rolling_mean_{window}'] = df[col].rolling(window=window).mean()
            df_featured[f'{col}_rolling_std_{window}'] = df[col].rolling(window=window).std()
            df_featured[f'{col}_rolling_min_{window}'] = df[col].rolling(window=window).min()
            df_featured[f'{col}_rolling_max_{window}'] = df[col].rolling(window=window).max()
    
    # Trend features
    for col in df.select_dtypes(include=[np.number]).columns:
        df_featured[f'{col}_trend'] = df[col].diff()
        df_featured[f'{col}_pct_change'] = df[col].pct_change()
    
    # Fill NaN values
    df_featured = df_featured.fillna(method='bfill').fillna(method='ffill')
    
    print(f"Created additional features. New shape: {df_featured.shape}")
    return df_featured

## 2. XGBoost Model Training

In [ ]:
def train_xgboost_rul(X_train, y_train, config=None):
    """Train XGBoost model for RUL prediction"""
    if config is None:
        config = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'max_depth': 6,
            'learning_rate': 0.1,
            'n_estimators': 100,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42
        }
    
    model = xgb.XGBRegressor(**config)
    model.fit(X_train, y_train, verbose=True)
    
    return model

def hyperparameter_tuning(X_train, y_train, param_grid=None):
    """Perform hyperparameter tuning for XGBoost"""
    if param_grid is None:
        param_grid = {
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [50, 100, 200],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        cv=3,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {-grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_

## 3. Model Evaluation

In [ ]:
def evaluate_rul_model(model, X_test, y_test):
    """Evaluate RUL prediction model"""
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print("Model Evaluation Metrics:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    
    # Calculate percentage errors
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"MAPE: {mape:.2f}%")
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'predictions': y_pred,
        'actual': y_test
    }

def cross_validate_model(model, X, y, cv=5):
    """Perform cross-validation"""
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    
    print(f"Cross-validation RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
    
    return rmse_scores

## 4. Visualization Functions

In [ ]:
def plot_predictions_vs_actual(y_true, y_pred, title="RUL Predictions vs Actual"):
    """Plot predicted vs actual RUL values"""
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.scatter(y_true, y_pred, alpha=0.6, color='blue')
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('Actual RUL')
    plt.ylabel('Predicted RUL')
    plt.title('Predicted vs Actual RUL')
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    errors = y_pred - y_true
    plt.hist(errors, bins=50, alpha=0.7, color='green')
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    plt.title('Prediction Error Distribution')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    plt.plot(y_true[:100], label='Actual', alpha=0.7)
    plt.plot(y_pred[:100], label='Predicted', alpha=0.7)
    plt.xlabel('Sample Index')
    plt.ylabel('RUL')
    plt.title('RUL Over Time (First 100 samples)')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 2, 4)
    plt.plot(np.abs(errors)[:100], color='red', alpha=0.7)
    plt.xlabel('Sample Index')
    plt.ylabel('Absolute Error')
    plt.title('Absolute Prediction Error Over Time')
    plt.grid(True)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

def plot_feature_importance(model, feature_names=None, top_n=20):
    """Plot feature importance"""
    if feature_names is None:
        feature_names = [f'feature_{i}' for i in range(len(model.feature_importances_))]
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(1, 2, 1)
    importance_df.head(top_n).plot.barh(x='feature', y='importance', ax=plt.gca())
    plt.title(f'Top {top_n} Feature Importances')
    plt.xlabel('Importance')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(len(importance_df)), importance_df['importance'].values)
    plt.xlabel('Feature Rank')
    plt.ylabel('Importance')
    plt.title('Feature Importance Distribution')
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    return importance_df

## 5. SHAP Explainability

In [ ]:
def explain_model_with_shap(model, X_train, X_test, feature_names=None, max_evals=1000):
    """Use SHAP to explain model predictions"""
    try:
        # Create SHAP explainer
        explainer = shap.TreeExplainer(model)
        
        # Calculate SHAP values for test set (sample for efficiency)
        X_sample = X_test[:min(max_evals, len(X_test))]
        shap_values = explainer.shap_values(X_sample)
        
        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample, feature_names=feature_names, show=False)
        plt.title('SHAP Feature Importance Summary')
        plt.tight_layout()
        plt.show()
        
        # Waterfall plot for first prediction
        if len(X_sample) > 0:
            plt.figure(figsize=(10, 6))
            shap.plots.waterfall(explainer.expected_value, shap_values[0], X_sample[0], 
                               feature_names=feature_names, show=False)
            plt.title('SHAP Waterfall Plot for First Prediction')
            plt.tight_layout()
            plt.show()
        
        return explainer, shap_values
        
    except Exception as e:
        print(f"SHAP analysis failed: {e}")
        return None, None

## 6. Complete Training Pipeline

In [ ]:
def train_xgboost_rul_pipeline(data, config=None, tune_hyperparams=False):
    """Complete XGBoost RUL training pipeline"""
    if config is None:
        config = {
            'sequence_length': 30,
            'test_size': 0.2,
            'random_state': 42,
            'tune_hyperparams': tune_hyperparams
        }
    
    # Prepare data
    print("Preparing data for RUL prediction...")
    X, y = prepare_rul_data(data, sequence_length=config['sequence_length'])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=config['test_size'], random_state=config['random_state']
    )
    
    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    
    # Train model
    if tune_hyperparams:
        print("Performing hyperparameter tuning...")
        model = hyperparameter_tuning(X_train, y_train)
    else:
        print("Training XGBoost model...")
        model = train_xgboost_rul(X_train, y_train)
    
    # Evaluate model
    print("Evaluating model...")
    eval_results = evaluate_rul_model(model, X_test, y_test)
    
    # Cross-validation
    print("Performing cross-validation...")
    cv_scores = cross_validate_model(model, X_train, y_train)
    
    # Visualizations
    plot_predictions_vs_actual(y_test, eval_results['predictions'])
    
    # Feature importance
    importance_df = plot_feature_importance(model)
    
    # SHAP analysis
    print("Performing SHAP analysis...")
    explainer, shap_values = explain_model_with_shap(model, X_train, X_test)
    
    return {
        'model': model,
        'evaluation': eval_results,
        'cv_scores': cv_scores,
        'feature_importance': importance_df,
        'explainer': explainer,
        'shap_values': shap_values,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    }

## 7. Model Saving and Loading

In [ ]:
def save_xgboost_model(model, filepath):
    """Save trained XGBoost model"""
    model.save_model(filepath)
    print(f"Model saved to {filepath}")

def load_xgboost_model(filepath):
    """Load trained XGBoost model"""
    model = xgb.XGBRegressor()
    model.load_model(filepath)
    print(f"Model loaded from {filepath}")
    return model

## 8. Example Usage

In [ ]:
# Example usage (uncomment and modify for your data)
# 
# # Load and preprocess your data
# # data = ... # Your preprocessed time series data (pandas DataFrame)
# 
# # Train RUL prediction model
# results = train_xgboost_rul_pipeline(data, tune_hyperparams=False)
# 
# # Save model
# save_xgboost_model(results['model'], 'models/xgboost_rul_model.json')
# 
# # For inference on new data:
# # new_X, _ = prepare_rul_data(new_data, sequence_length=30)
# # rul_predictions = results['model'].predict(new_X)

print("XGBoost RUL training functions defined. Ready to use!")